In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# pandas global settings
pd.options.display.max_columns=200

We've talked about Random Forests. Now it's time to build one.

Here we'll use data from Lending Club to predict the state of a loan given some information about it. You can find the dataset [here](https://www.lendingclub.com/info/download-data.action). We'll use 2015 data. ([Thinkful mirror](https://www.dropbox.com/s/m7z42lubaiory33/LoanStats3d.csv?dl=0))

In [3]:
path="../../../../Datafiles/LendingClub/"
file = 'LoanStats3d.csv'
url = 'https://resources.lendingclub.com/LoanStats3d.csv'

print("We are working with 2015 Lending dataset from {}".format(url))
y2015 = pd.read_csv(path + file, header=1, skipfooter=4, sep=',', quotechar='"', engine='python')
print("there are {:,} entries in the data frame, with {} columns, from data file: {}".format(len(y2015),
                                                                                             len(y2015.columns),                                                                                            file))
y2015.head(20)
print("y2015 has {} rows.".format(len(y2015)))

We are working with 2015 Lending dataset from https://resources.lendingclub.com/LoanStats3d.csv
there are 421,095 entries in the data frame, with 111 columns, from data file: LoanStats3d.csv
y2015 has 421095 rows.


In [4]:
y2015.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139,16000,16000,16000.0,60 months,14.85%,379.39,C,C5,Bookkeeper/Accounting,10+ years,MORTGAGE,48000.0,Not Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,credit_card,Credit card refinancing,297xx,SC,33.18,0,Jun-1991,0,33.0,2.0,11,2,19108,29.6%,19,w,13668.88,13668.88,4519.68,4519.68,2331.12,2188.56,0.0,0.0,0.0,Jan-2017,379.39,Jan-2017,Jan-2017,0,33.0,1,INDIVIDUAL,NaN,NaN,NaN,0,0,31329,1.0,2.0,1.0,2.0,6.0,12221.0,91.0,1.0,4.0,14982.0,65.0,284700,0.0,0.0,1.0,6,2848,263953.0,17.6,0,0,131.0,294,11,6,2,11.0,76.0,11.0,76.0,2,6,9,6,8,6,9,11.0,9,11,0.0,0,0,2,78.9,0.0,0,2,298100,31329,281300,13400
1,68354783,73244544,9600,9600,9600.0,36 months,7.49%,298.58,A,A4,tech,8 years,MORTGAGE,60000.0,Not Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,credit_card,Credit card refinancing,299xx,SC,22.44,0,Jun-1996,0,NaN,NaN,7,0,7722,59.4%,9,w,6635.69,6635.69,3572.97,3572.97,2964.31,608.66,0.0,0.0,0.0,Jan-2017,298.58,Jan-2017,Jan-2017,0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0,55387,0.0,3.0,0.0,0.0,47.0,47665.0,43.0,2.0,2.0,4744.0,50.0,13000,0.0,0.0,2.0,2,7912,4778.0,61.8,0,0,234.0,91,9,9,0,11.0,NaN,9.0,NaN,0,3,3,3,3,5,4,4.0,3,7,0.0,0,0,2,100.0,66.7,0,0,88635,55387,12500,75635
2,68466916,73356753,25000,25000,25000.0,36 months,7.49%,777.55,A,A4,Sales Manager,10+ years,MORTGAGE,109000.0,Not Verified,Dec-2015,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,226xx,VA,26.02,0,Dec-2001,1,NaN,NaN,9,0,20862,54.3%,19,w,0.00,0.00,26224.23,26224.23,25000.00,1224.23,0.0,0.0,0.0,Sep-2016,20807.39,NaN,Jan-2017,0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0,305781,0.0,3.0,0.0,1.0,13.0,47194.0,58.0,0.0,1.0,8937.0,57.0,38400,1.0,0.0,1.0,2,33976,17538.0,54.3,0,0,142.0,168,13,13,3,13.0,NaN,0.0,NaN,0,3,3,5,6,7,5,9.0,3,9,0.0,0,0,0,100.0,20.0,0,0,373572,68056,38400,82117
3,68466961,73356799,28000,28000,28000.0,36 months,6.49%,858.05,A,A2,Senior Manager,10+ years,MORTGAGE,92000.0,Not Verified,Dec-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,275xx,NC,21.60,0,May-1984,0,42.0,NaN,16,0,51507,64.5%,24,w,19263.77,19263.77,10271.36,10271.36,8736.23,1535.13,0.0,0.0,0.0,Jan-2017,858.05,Jan-2017,Jan-2017,0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0,221110,0.0,2.0,0.0,0.0,27.0,23413.0,55.0,0.0,1.0,8752.0,61.0,79900,0.0,1.0,0.0,1,13819,16623.0,59.9,0,0,82.0,379,19,19,

In [5]:
y2015.tail(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421090,36371250,39102635,10000,10000,10000.0,36 months,11.99%,332.10,B,B5,Office Assistant,8 years,RENT,31000.0,Verified,Jan-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,905xx,CA,28.69,0,Sep-2004,0,NaN,77.0,9,1,14037,82.1%,15,f,3737.83,3737.83,7962.24,7962.24,6262.17,1700.07,0.0,0.00,0.0000,Jan-2017,332.10,Feb-2017,Jan-2017,0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0,25274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17100,NaN,NaN,NaN,2,3159,448.0,95.1,0,0,122.0,115,17,10,0,90.0,NaN,9.0,NaN,0,3,6,3,4,7,7,8.0,6,9,0.0,0,0,1,100.0,100.0,0,0,32950,25274,9200,15850
421091,36441262,39152692,24000,24000,24000.0,36 months,11.99%,797.03,B,B5,Director of Maintenance,10+ years,MORTGAGE,79000.0,Verified,Jan-2015,Current,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,home_improvement,Home improvement,072xx,NJ,3.90,0,Mar-1974,1,26.0,NaN,5,0,8621,84.5%,23,f,8971.06,8971.06,19096.75,19096.75,15028.94,4067.81,0.0,0.00,0.0000,Jan-2017,797.03,Feb-2017,Jan-2017,0,29.0,1,INDIVIDUAL,NaN,NaN,NaN,0,0,140285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10200,NaN,NaN,NaN,3,35071,976.0,89.2,0,0,NaN,488,3,3,4,3.0,29.0,3.0,29.0,10,2,3,3,14,0,4,17.0,3,5,0.0,0,0,2,56.5,100.0,0,0,152650,8621,9000,0
421092,36271333,38982739,13000,13000,13000.0,60 months,15.99%,316.07,D,D2,Radiologist Technologist,5 years,RENT,35000.0,Verified,Jan-2015,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,378xx,TN,30.90,0,Sep-2003,0,NaN,84.0,9,1,11031,61.3%,22,w,0.00,0.00,5034.02,5034.02,2527.70,2506.32,0.0,0.00,0.0000,May-2016,316.07,NaN,Dec-2016,0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0,34178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18000,NaN,NaN,NaN,7,3798,2554.0,75.9,0,0,134.0,108,6,6,0,17.0,NaN,10.0,NaN,0,4,6,4,4,12,7,10.0,6,9,0.0,0,0,3,100.0,50.0,1,0,51239,34178,10600,33239
421093,36490806,39222577,12000,12000,12000.0,60 months,19.99%,317.86,E,E3,Painter,1 year,RENT,64400.0,Source Verified,Jan-2015,Charged Off,n,https://lendingclub.com/browse/loanDetail.acti...,NaN,debt_consolidation,Debt consolidation,010xx,MA,27.19,1,Oct-2003,2,22.0,NaN,17,0,8254,30.6%,20,w,0.00,0.00,5366.99,5366.99,1552.69,2234.98,0.0,1579.32,284.2776,Jan-2016,317.86,NaN,Oct-2016,1,22.0,1,INDIVIDUAL,NaN,NaN,NaN,0,0,58418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27000,NaN,NaN,NaN,6,3895,3752.0,61.3,0,0,56.0,83,5,5,0,12.0,NaN,0.0,NaN,1,5,7,7,7,3,1

In [6]:
for col in y2015.columns:
    print("column: {}".format(col))

column: id
column: member_id
column: loan_amnt
column: funded_amnt
column: funded_amnt_inv
column: term
column: int_rate
column: installment
column: grade
column: sub_grade
column: emp_title
column: emp_length
column: home_ownership
column: annual_inc
column: verification_status
column: issue_d
column: loan_status
column: pymnt_plan
column: url
column: desc
column: purpose
column: title
column: zip_code
column: addr_state
column: dti
column: delinq_2yrs
column: earliest_cr_line
column: inq_last_6mths
column: mths_since_last_delinq
column: mths_since_last_record
column: open_acc
column: pub_rec
column: revol_bal
column: revol_util
column: total_acc
column: initial_list_status
column: out_prncp
column: out_prncp_inv
column: total_pymnt
column: total_pymnt_inv
column: total_rec_prncp
column: total_rec_int
column: total_rec_late_fee
column: recoveries
column: collection_recovery_fee
column: last_pymnt_d
column: last_pymnt_amnt
column: next_pymnt_d
column: last_credit_pull_d
column: collect

## The Blind Approach

Now, as we've seen before, creating a model is the easy part. Let's try just using everything we've got and throwing it without much thought into a Random Forest. SKLearn requires the independent variables to be be numeric, and all we want is dummy variables so let's use `get_dummies` from Pandas to generate a dummy variable for every categorical colummn and see what happens off of this kind of naive approach.

In [7]:
# from sklearn import ensemble
# from sklearn.model_selection import cross_val_score

# rfc = ensemble.RandomForestClassifier()
# X = y2015.drop('loan_status', 1)
# Y = y2015['loan_status']
# X = pd.get_dummies(X)

# cross_val_score(rfc, X, Y, cv=5)

Did your kernel die? My kernel died.

Guess it isn't always going to be that easy...

Can you think of what went wrong?

(You're going to have to reset your kernel and reload the column, BUT DON'T RUN THE MODEL AGAIN OR YOU'LL CRASH THE KERNEL AGAIN!)

## Data Cleaning

Well, `get_dummies` can be a very memory intensive thing, particularly if data are typed poorly. We got a warning about that earlier. Mixed data types get converted to objects, and that could create huge problems. Our dataset is about 400,000 rows. If there's a bad type there its going to see 400,000 distinct values and try to create dummies for all of them. That's bad. Lets look at all our categorical variables and see how many distinct counts there are...

In [8]:
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i)
    print(column.nunique())

term
2
int_rate
110
grade
7
sub_grade
35
emp_title
120812
emp_length
11
home_ownership
4
verification_status
3
issue_d
12
loan_status
7
pymnt_plan
1
url
421095
desc
34
purpose
14
title
27
zip_code
914
addr_state
49
earliest_cr_line
668
revol_util
1211
initial_list_status
2
last_pymnt_d
25
next_pymnt_d
4
last_credit_pull_d
26
application_type
2
verification_status_joint
3


Well that right there is what's called a problem. Some of these have over a hundred thousand distinct types. Lets drop the ones with over 30 unique values, converting to numeric where it makes sense. In doing this there's a lot of code that gets written to just see if the numeric conversion makes sense. It's a manual process that we'll abstract away and just include the conversion.

You could extract numeric features from the dates, but here we'll just drop them. There's a lot of data, it shouldn't be a huge problem.

In [9]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

Wonder what was causing the dtype error on the id column, which _should_ have all been integers? Let's look at the end of the file.

In [10]:
y2015.tail()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
421090,36371250,39102635,10000,10000,10000.0,36 months,11.99,332.10,B,8 years,RENT,31000.0,Verified,Jan-2015,Current,n,debt_consolidation,Debt consolidation,28.69,0,0,NaN,77.0,9,1,14037,15,f,3737.83,3737.83,7962.24,7962.24,6262.17,1700.07,0.0,0.00,0.0000,Jan-2017,332.10,Feb-2017,Jan-2017,0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0,25274,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17100,NaN,NaN,NaN,2,3159,448.0,95.1,0,0,122.0,115,17,10,0,90.0,NaN,9.0,NaN,0,3,6,3,4,7,7,8.0,6,9,0.0,0,0,1,100.0,100.0,0,0,32950,25274,9200,15850
421091,36441262,39152692,24000,24000,24000.0,36 months,11.99,797.03,B,10+ years,MORTGAGE,79000.0,Verified,Jan-2015,Current,n,home_improvement,Home improvement,3.90,0,1,26.0,NaN,5,0,8621,23,f,8971.06,8971.06,19096.75,19096.75,15028.94,4067.81,0.0,0.00,0.0000,Jan-2017,797.03,Feb-2017,Jan-2017,0,29.0,1,INDIVIDUAL,NaN,NaN,NaN,0,0,140285,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10200,NaN,NaN,NaN,3,35071,976.0,89.2,0,0,NaN,488,3,3,4,3.0,29.0,3.0,29.0,10,2,3,3,14,0,4,17.0,3,5,0.0,0,0,2,56.5,100.0,0,0,152650,8621,9000,0
421092,36271333,38982739,13000,13000,13000.0,60 months,15.99,316.07,D,5 years,RENT,35000.0,Verified,Jan-2015,Charged Off,n,debt_consolidation,Debt consolidation,30.90,0,0,NaN,84.0,9,1,11031,22,w,0.00,0.00,5034.02,5034.02,2527.70,2506.32,0.0,0.00,0.0000,May-2016,316.07,NaN,Dec-2016,0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0,34178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18000,NaN,NaN,NaN,7,3798,2554.0,75.9,0,0,134.0,108,6,6,0,17.0,NaN,10.0,NaN,0,4,6,4,4,12,7,10.0,6,9,0.0,0,0,3,100.0,50.0,1,0,51239,34178,10600,33239
421093,36490806,39222577,12000,12000,12000.0,60 months,19.99,317.86,E,1 year,RENT,64400.0,Source Verified,Jan-2015,Charged Off,n,debt_consolidation,Debt consolidation,27.19,1,2,22.0,NaN,17,0,8254,20,w,0.00,0.00,5366.99,5366.99,1552.69,2234.98,0.0,1579.32,284.2776,Jan-2016,317.86,NaN,Oct-2016,1,22.0,1,INDIVIDUAL,NaN,NaN,NaN,0,0,58418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27000,NaN,NaN,NaN,6,3895,3752.0,61.3,0,0,56.0,83,5,5,0,12.0,NaN,0.0,NaN,1,5,7,7,7,3,15,16.0,7,17,0.0,0,1,2,95.0,66.7,0,0,96919,58418,9700,69919
421094,36271262,38982659,20000,20000,20000.0,36 months,11.99,664.20,B,10+ years,RENT,100000.0,Verified,Jan-2015,Current,n,credit_card,Credit card refinancing,10.83,0,1,NaN,110.0,8,1,33266,10,f,7475.67,7475.67,15966.05,15966.05,12524.33,3441.72,0.0,0.00,0.0000,Jan-2017,664.20,Feb-2017,Jan-2017,0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0,33307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41700,NaN,NaN,NaN,1,4163,8434.0,79.8,0,0,

In [6]:
# Remove two summary rows at the end that don't actually contain data.
# y2015 = y2015[:-2]
# skipfooter=4 in read_csv fixed this better...

Now this should be better. Let's try again.

In [11]:
pd.get_dummies(y2015)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,term_ 36 months,term_ 60 months,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,emp_length_1 year,emp_length_10+ years,emp_length_2 years,emp_length_3 years,...,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,title_Business,title_Car financing,title_Credit Card/Auto Repair,title_Credit card refinancing,title_Debt consolidation,title_DebtC,title_Green loan,title_Home buying,title_Home improvement,title_Learning and training,title_Major purchase,title_Medical expenses,title_Moving and relocation,title_New Baby and New House (CC Consolidate),title_Other,title_Pay off Lowes Card,title_Paying off higher interest cards & auto,title_Prescription Drug and Medical Costs,title_SAVE,title_Simple Loan Until Contract Is Completed,title_Student Loan,title_Trying to come back to reality!,title_Vacation,title_considerate,title_new day,title_new kitchen for momma!,title_odymeds,initial_list_status_f,initial_list_status_w,last_pymnt_d_Apr-2015,last_pymnt_d_Apr-2016,last_pymnt_d_Aug-2015,last_pymnt_d_Aug-2016,last_pymnt_d_Dec-2015,last_pymnt_d_Dec-2016,last_pymnt_d_Feb-2015,last_pymnt_d_Feb-2016,last_pymnt_d_Jan-2015,last_pymnt_d_Jan-2016,last_pymnt_d_Jan-2017,last_pymnt_d_Jul-2015,last_pymnt_d_Jul-2016,last_pymnt_d_Jun-2015,last_pymnt_d_Jun-2016,last_pymnt_d_Mar-2015,last_pymnt_d_Mar-2016,last_pymnt_d_May-2015,last_pymnt_d_May-2016,last_pymnt_d_Nov-2015,last_pymnt_d_Nov-2016,last_pymnt_d_Oct-2015,last_pymnt_d_Oct-2016,last_pymnt_d_Sep-2015,last_pymnt_d_Sep-2016,next_pymnt_d_Feb-2017,next_pymnt_d_Jan-2017,next_pymnt_d_Jul-2016,next_pymnt_d_Mar-2017,last_credit_pull_d_Apr-2015,last_credit_pull_d_Apr-2016,last_credit_pull_d_Aug-2015,last_credit_pull_d_Aug-2016,last_credit_pull_d_Dec-2014,last_credit_pull_d_Dec-2015,last_credit_pull_d_Dec-2016,last_credit_pull_d_Feb-2015,last_credit_pull_d_Feb-2016,last_credit_pull_d_Jan-2015,last_credit_pull_d_Jan-2016,last_credit_pull_d_Jan-2017,last_credit_pull_d_Jul-2015,last_credit_pull_d_Jul-2016,last_credit_pull_d_Jun-2015,last_credit_pull_d_Jun-2016,last_credit_pull_d_Mar-2015,last_credit_pull_d_Mar-2016,last_credit_pull_d_May-2015,last_credit_pull_d_May-2016,last_credit_pull_d_Nov-2015,last_credit_pull_d_Nov-2016,last_credit_pull_d_Oct-2015,last_credit_pull_d_Oct-2016,last_credit_pull_d_Sep-2015,last_credit_pull_d_Sep-2016,application_type_INDIVIDUAL,application_type_JOINT,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
0,68009401,72868139,16000,16000,16000.0,14.85,379.39,48000.0,33.18,0,0,33.0,2.0,11,2,19108

In [12]:
y2015.head(5)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,68009401,72868139,16000,16000,16000.0,60 months,14.85,379.39,C,10+ years,MORTGAGE,48000.0,Not Verified,Dec-2015,Current,n,credit_card,Credit card refinancing,33.18,0,0,33.0,2.0,11,2,19108,19,w,13668.88,13668.88,4519.68,4519.68,2331.12,2188.56,0.0,0.0,0.0,Jan-2017,379.39,Jan-2017,Jan-2017,0,33.0,1,INDIVIDUAL,NaN,NaN,NaN,0,0,31329,1.0,2.0,1.0,2.0,6.0,12221.0,91.0,1.0,4.0,14982.0,65.0,284700,0.0,0.0,1.0,6,2848,263953.0,17.6,0,0,131.0,294,11,6,2,11.0,76.0,11.0,76.0,2,6,9,6,8,6,9,11.0,9,11,0.0,0,0,2,78.9,0.0,0,2,298100,31329,281300,13400
1,68354783,73244544,9600,9600,9600.0,36 months,7.49,298.58,A,8 years,MORTGAGE,60000.0,Not Verified,Dec-2015,Current,n,credit_card,Credit card refinancing,22.44,0,0,NaN,NaN,7,0,7722,9,w,6635.69,6635.69,3572.97,3572.97,2964.31,608.66,0.0,0.0,0.0,Jan-2017,298.58,Jan-2017,Jan-2017,0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0,55387,0.0,3.0,0.0,0.0,47.0,47665.0,43.0,2.0,2.0,4744.0,50.0,13000,0.0,0.0,2.0,2,7912,4778.0,61.8,0,0,234.0,91,9,9,0,11.0,NaN,9.0,NaN,0,3,3,3,3,5,4,4.0,3,7,0.0,0,0,2,100.0,66.7,0,0,88635,55387,12500,75635
2,68466916,73356753,25000,25000,25000.0,36 months,7.49,777.55,A,10+ years,MORTGAGE,109000.0,Not Verified,Dec-2015,Fully Paid,n,debt_consolidation,Debt consolidation,26.02,0,1,NaN,NaN,9,0,20862,19,w,0.00,0.00,26224.23,26224.23,25000.00,1224.23,0.0,0.0,0.0,Sep-2016,20807.39,NaN,Jan-2017,0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0,305781,0.0,3.0,0.0,1.0,13.0,47194.0,58.0,0.0,1.0,8937.0,57.0,38400,1.0,0.0,1.0,2,33976,17538.0,54.3,0,0,142.0,168,13,13,3,13.0,NaN,0.0,NaN,0,3,3,5,6,7,5,9.0,3,9,0.0,0,0,0,100.0,20.0,0,0,373572,68056,38400,82117
3,68466961,73356799,28000,28000,28000.0,36 months,6.49,858.05,A,10+ years,MORTGAGE,92000.0,Not Verified,Dec-2015,Current,n,debt_consolidation,Debt consolidation,21.60,0,0,42.0,NaN,16,0,51507,24,w,19263.77,19263.77,10271.36,10271.36,8736.23,1535.13,0.0,0.0,0.0,Jan-2017,858.05,Jan-2017,Jan-2017,0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0,221110,0.0,2.0,0.0,0.0,27.0,23413.0,55.0,0.0,1.0,8752.0,61.0,79900,0.0,1.0,0.0,1,13819,16623.0,59.9,0,0,82.0,379,19,19,2,19.0,48.0,NaN,42.0,0,7,9,9,11,4,13,18.0,9,16,0.0,0,0,0,91.7,22.2,0,0,304003,74920,41500,42503
4,68495092,73384866,8650,8650,8650.0,36 months,19.89,320.99,E,8 years,RENT,55000.0,Verified,Dec-2015,Fully Paid,n,debt_consolidation,Debt consolidation,25.49,0,4,NaN,30.0,18,1,9568,19,w,0.00,0.00,9190.49,9190.49,8650.00,540.49,0.0,0.0,0.0,May-2016,8251.42,NaN,Jun-2016,0,NaN,1,INDIVIDUAL,NaN,NaN,NaN,0,0,18926,6.0,1.0,0.0,1.0,19.0,9358.0,51.0,12.0,16.0,653.0,46.0,2

It finally works! We had to sacrifice sub grade, state address and description, but that's fine. If you want to include them you could run the dummies independently and then append them back to the dataframe.

## Second Attempt

Now let's try this model again.

We're also going to drop NA columns, rather than impute, because our data is rich enough that we can probably get away with it.

This model may take a few minutes to run.

In [13]:
print("this takes about 4 minutes to run, just so you know, on my Mac.")
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

cross_val_score(rfc, X, Y, cv=10)

/Users/lou/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lou/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lou/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lou/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/lou/anaconda3/lib/python3.7/site-packages/skl

array([0.9794353 , 0.98033768, 0.97651445, 0.98154876, 0.97435289,
       0.9706958 , 0.96611176, 0.9802655 , 0.9793621 , 0.98057284])

The score cross validation reports is the accuracy of the tree. Here we're about 98% accurate.

That works pretty well, but there are a few potential problems. Firstly, we didn't really do much in the way of feature selection or model refinement. As such there are a lot of features in there that we don't really need. Some of them are actually quite impressively useless.

There's also some variance in the scores. The fact that one gave us only 93% accuracy while others gave higher than 98 is concerning. This variance could be corrected by increasing the number of estimators. That will make it take even longer to run, however, and it is already quite slow.

## DRILL: Third Attempt

So here's your task. Get rid of as much data as possible without dropping below an average of 90% accuracy in a 10-fold cross validation.

You'll want to do a few things in this process. First, dive into the data that we have and see which features are most important. This can be the raw features or the generated dummies. You may want to use PCA or correlation matrices.

Can you do it without using anything related to payment amount or outstanding principal? How do you know?

In [19]:
# X.sample(20)
#########################
#   Best Features stuff #
#########################
#https://www.kaggle.com/iabhishekofficial/mobile-price-classification#train.csv
if x = -99:
    import pandas as pd
    import numpy as np
    from sklearn.feature_selection import SelectKBest
    from sklearn.feature_selection import chi2
    
    data = pd.read_csv("D://Blogs//train.csv")
    X = data.iloc[:,0:20]  #independent columns
    y = data.iloc[:,-1]    #target column i.e price range
    
    #apply SelectKBest class to extract top 10 best features
    bestfeatures = SelectKBest(score_func=chi2, k=10)
    fit = bestfeatures.fit(X,y)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(X.columns)
    #concat two dataframes for better visualization 
    featureScores = pd.concat([dfcolumns,dfscores],axis=1)
    featureScores.columns = ['Specs','Score']  #naming the dataframe columns
    print(featureScores.nlargest(10,'Score'))  #print 10 best features

In [29]:
# Your code here.
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

rfc = ensemble.RandomForestClassifier()
X = y2015.drop('loan_status', 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

# cross_val_score(rfc, X, Y, cv=10)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=0)
clf = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)

clf.fit(X_train, y_train)

#print the name and gini importances of each feature
for feature in zip(df.columns, clf.feature_importances_):
    print(feature)


this takes about 4 minutes to run, just so you know, on my Mac.


NameError: name 'df' is not defined